In [1]:
from SBM_SDE import *
from obs_and_flow_classes_and_functions import *
from get_CO2 import *
import seaborn as sns
import torch
from torch import nn
import torch.distributions as d
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [2]:
torch.manual_seed(1)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

cuda_id = 1
dt = .2 #SDE discretization timestep.
t = 100 #Simulation run for T hours.
n = int(t / dt) 
t_span = np.linspace(0, t, n + 1)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
l_r = 1e-3
niter = 21
piter = 10
batch_size = 2 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.

In [3]:
temp_ref = 283

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_CO2 = 0.0001
s_SOC = 0.01
s_DOC = 0.01
s_MBC = 0.01

SCON_C_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}
SCON_SS_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC}

In [4]:
#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.01
s_DOC = 0.01
s_MBC = 0.01
s_EEC = 0.01

SAWB_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [5]:
#System parameters from deterministic model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.01
s_DOC = 0.01
s_MBC = 0.01
s_EEC = 0.01

SAWB_ECA_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_ECA_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [6]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
i_s_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous SOC input function
i_d_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous DOC input function

In [7]:
def neg_log_lik(C_PATH, T_SPAN_TENSOR, DT, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF):
    drift, diffusion_sqrt = DRIFT_DIFFUSION(C_PATH[:, :-1, :], T_SPAN_TENSOR[:, :-1, :], I_S_TENSOR[:, :-1, :], I_D_TENSOR[:, :-1, :], PARAMS_DICT, TEMP_GEN, TEMP_REF)
    euler_maruyama_state_sample_object = d.multivariate_normal.MultivariateNormal(loc = C_PATH[:, :-1, :] + drift * DT, scale_tril = diffusion_sqrt * math.sqrt(DT))
    return -euler_maruyama_state_sample_object.log_prob(C_PATH[:, 1:, :]).sum(-1)

In [8]:
def train(DEVICE, L_R, NITER, PRETRAIN_ITER, BATCH_SIZE, ObsModel, csv_to_obs_df, DATA_CSV, OBS_ERROR_SCALE, STATE_DIM, T, DT, N, T_SPAN_TENSOR, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF, ANALYTICAL_STEADY_STATE_INIT, GET_CO2):
    obs_times, obs_means, obs_error = csv_to_obs_df(DATA_CSV, STATE_DIM + 1, T, OBS_ERROR_SCALE) #Need to +1 because data has CO2 observations on top of other state observations. 
    obs_model_no_CO2 = ObsModel(DEVICE, obs_times, DT, obs_means[:-1, :], obs_error[:, :-1])
    obs_model_CO2 = ObsModel(DEVICE, obs_times, DT, obs_means, obs_error)
    net = SDEFlow(DEVICE, BATCH_SIZE, obs_model_no_CO2, STATE_DIM, T, DT, N).to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr = L_R) 
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    best_loss_norm = 1e10
    best_loss_ELBO = 1e20
    norm_losses = [best_loss_norm] * 10
    ELBO_losses = [best_loss_ELBO] * 10
    C0 = ANALYTICAL_STEADY_STATE_INIT(I_S_TENSOR[0, 0, 0].item(), I_D_TENSOR[0, 0, 0].item(), PARAMS_DICT) #Calculate deterministic initial conditions.
    C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE) #Assign initial conditions to C_PATH.
    with tqdm(total = NITER, desc = f'Train Diffusion', position = -1) as tq:
        for iter in range(NITER):
            net.train()
            optimizer.zero_grad()
            C_PATH, log_prob = net() #Obtain paths with solutions at times after t0.
            C_PATH = torch.cat([C0, C_PATH], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            CO2 = GET_CO2(C_PATH, T_SPAN_TENSOR, PARAMS_DICT, TEMP_GEN, TEMP_REF)
            x_with_CO2 = torch.cat([C_PATH, CO2], -1)
            if iter <= PRETRAIN_ITER:
                l1_norm_element = x_with_CO2 - torch.mean(obs_model_CO2.mu, -1)
                l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l1_norm.item())
                #l2_norm_element = C_PATH - torch.mean(OBS_MODEL.mu, -1)
                #l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                #best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                #l2_norm.backward()
                #norm_losses.append(l2_norm.item())
                if len(norm_losses) > 10:
                    norm_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average norm loss at {iter} iterations is: {sum(norm_losses) / len(norm_losses)}. Best norm loss value is: {best_loss_norm}.")
                    print('\nx with CO2 means across time =', x_with_CO2.mean(-2))
                    print('\nx with CO2 =', x_with_CO2)
                l1_norm.backward()
            else:
                log_lik = neg_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), dt, I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE), DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF)
                ELBO = log_prob.mean() + log_lik.mean() - obs_model_CO2(x_with_CO2) #obs_model_CO2(x_with_CO2) is obs log likelihood.
                best_loss_ELBO = ELBO if ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(ELBO.item())
                if len(ELBO_losses) > 10:
                    ELBO_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average ELBO loss at {iter} iterations is: {sum(ELBO_losses) / len(ELBO_losses)}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nx with CO2 means across time =', x_with_CO2.mean(-2))
                    print('\nx with CO2 =', x_with_CO2)
                ELBO.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            optimizer.step()
            if iter % 100000 == 0 and iter > 0:
                optimizer.param_groups[0]['lr'] *= 0.1
            tq.update()

In [9]:
train(devi, l_r, niter, piter, batch_size, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON, get_CO2_CON)


Train Diffusion:   0%|          | 0/21 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000004606.148438. Best norm loss value is: 46061.484375.

x with CO2 means across time = tensor([[1.0453, 0.8656, 1.0082, 0.0036],
        [1.0373, 0.9053, 1.0472, 0.0038]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [4.7698e-01, 6.4249e-01, 7.9870e-01, 2.3585e-03],
         [1.0211e+00, 7.1676e-01, 8.1451e-01, 2.7343e-03],
         ...,
         [5.5339e-01, 7.8081e-01, 8.9842e-01, 5.2194e-03],
         [8.2167e-01, 9.2535e-01, 8.4106e-01, 6.2667e-03],
         [7.0344e-01, 8.3380e-01, 1.0603e+00, 5.8359e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [6.8004e-01, 7.4253e-01, 4.7636e-01, 2.6657e-03],
         [1.0980e+00, 7.3170e-01, 2.8954e-01, 2.7138e-03],
         ...,
         [1.2606e+00, 3.3355e-01, 3.5936e-01, 2.2683e-03],
         [8.8482e-01, 2.2890e-01, 2.6837e+00, 2.2242e-03],
         [1.4702e+00, 2.8612e-01, 1.8234e+00, 2.4465e-03]]],
 


Train Diffusion:  48%|████▊     | 10/21 [00:04<00:05,  2.07it/s]

Moving average norm loss at 10 iterations is: 42886.913671875. Best norm loss value is: 40646.66015625.

x with CO2 means across time = tensor([[5.1839e+00, 3.2652e-01, 5.2847e-01, 1.4705e-03],
        [5.8268e+00, 3.2795e-01, 5.3322e-01, 1.4881e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [7.1529e-01, 1.5719e+00, 5.2213e-01, 5.5626e-03],
         [2.6718e-01, 3.6928e+00, 3.0453e-01, 1.3429e-02],
         ...,
         [1.0188e-01, 3.8590e-01, 5.9272e-01, 2.6099e-03],
         [9.1500e+00, 4.6262e-01, 6.9826e-01, 3.6004e-03],
         [2.0786e-01, 4.4141e-01, 6.0533e-01, 3.0935e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [3.9525e-01, 5.8684e+00, 3.3217e-01, 2.0501e-02],
         [1.2679e+00, 8.3243e-01, 5.1674e-01, 3.1149e-03],
         ...,
         [7.0487e+00, 4.6411e-01, 6.4477e-01, 3.4251e-03],
         [8.8532e-01, 3.4912e-01, 5.1503e-01, 2.4417e-03],
         [4.5442e+00, 


Train Diffusion:  95%|█████████▌| 20/21 [00:09<00:00,  2.34it/s]

Moving average ELBO loss at 20 iterations is: 320460.7609375. Best ELBO loss value is: 138119.296875.

x with CO2 means across time = tensor([[1.8431, 0.9901, 2.7970, 0.0042],
        [1.9472, 1.0051, 3.0183, 0.0044]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [9.4777e-01, 1.8793e+00, 2.5561e+00, 6.9213e-03],
         [1.2469e+00, 1.6446e-01, 5.3275e-01, 6.9658e-04],
         ...,
         [1.2037e-01, 3.0575e-01, 7.2896e+00, 3.8048e-03],
         [2.2763e-01, 3.4305e-01, 9.5341e-01, 2.4864e-03],
         [1.9057e+00, 6.3482e-01, 4.2942e+00, 5.4307e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [2.1937e-01, 2.9278e-01, 9.6478e-01, 1.1585e-03],
         [2.4466e-02, 1.7421e+00, 3.2465e+00, 6.7832e-03],
         ...,
         [2.4684e+00, 2.9588e+00, 1.3493e+00, 1.9272e-02],
         [3.7970e+00, 3.3378e+00, 7.0128e+00, 2.3676e-02],
         [1.8069e-01, 3.0314e+00, 1.3617e+00, 2.0446e-02]]],
  


Train Diffusion: 100%|██████████| 21/21 [00:09<00:00,  2.20it/s]


In [10]:
train(devi, l_r, niter, piter, batch_size, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_SS, SCON_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON, get_CO2_CON)


Train Diffusion:   0%|          | 0/21 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000004559.75586. Best norm loss value is: 45597.55859375.

x with CO2 means across time = tensor([[0.8418, 0.6046, 0.7626, 0.0025],
        [0.8536, 0.6066, 0.7454, 0.0025]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [5.3415e-01, 5.5842e-01, 5.9027e-01, 2.0375e-03],
         [3.5351e-01, 6.7033e-01, 3.6241e-01, 2.4880e-03],
         ...,
         [8.3245e-01, 6.3657e-01, 7.9480e-01, 4.2878e-03],
         [9.5747e-01, 7.8242e-01, 8.4812e-01, 5.3462e-03],
         [6.7720e-01, 8.4948e-01, 1.0017e+00, 5.9230e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [5.3415e-01, 7.9098e-01, 6.7176e-01, 2.8592e-03],
         [4.1841e-01, 7.9560e-01, 7.4768e-01, 2.9989e-03],
         ...,
         [7.5762e-01, 7.0679e-01, 7.4594e-01, 4.7188e-03],
         [5.2685e-01, 5.0753e-01, 1.0011e+00, 3.5808e-03],
         [1.4583e+00, 5.2485e-01, 4.2726e-01, 3.6571e-03]]],



Train Diffusion:  48%|████▊     | 10/21 [00:04<00:04,  2.38it/s]

Moving average norm loss at 10 iterations is: 43797.5453125. Best norm loss value is: 42140.5546875.

x with CO2 means across time = tensor([[3.8620e+00, 1.8674e-01, 4.3455e-01, 8.7658e-04],
        [4.0918e+00, 1.8951e-01, 4.2876e-01, 8.9602e-04]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [4.1024e-01, 1.7868e+00, 2.5043e-01, 6.2682e-03],
         [1.2984e+00, 7.4351e-01, 2.8500e-01, 2.7597e-03],
         ...,
         [7.5998e+00, 2.5042e-01, 6.3811e-01, 2.0883e-03],
         [7.2570e+00, 2.0406e-01, 5.3344e-01, 1.7924e-03],
         [1.9419e+00, 2.4899e-01, 4.0683e-01, 1.8472e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [4.1024e-01, 5.2030e+00, 2.5835e-01, 1.8173e-02],
         [6.0584e-01, 6.6044e-01, 2.0237e-01, 2.4337e-03],
         ...,
         [3.0455e+00, 2.4235e-01, 9.1418e-01, 1.9076e-03],
         [9.8154e-01, 2.8950e-01, 6.5224e-01, 2.0944e-03],
         [6.8763e+00, 2.9


Train Diffusion:  95%|█████████▌| 20/21 [00:08<00:00,  2.39it/s]

Moving average ELBO loss at 20 iterations is: 1125770.0375. Best ELBO loss value is: 757522.75.

x with CO2 means across time = tensor([[5.1170e+00, 2.1840e-01, 2.7204e-01, 1.0054e-03],
        [5.2638e+00, 2.1656e-01, 2.7467e-01, 1.0084e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [5.5375e-01, 6.9852e-01, 1.0624e-01, 2.4586e-03],
         [7.6797e-01, 6.1640e-01, 7.3434e-02, 2.2585e-03],
         ...,
         [7.6071e+00, 1.9221e-01, 6.0159e-01, 1.7090e-03],
         [1.0211e+00, 1.5420e-01, 4.2144e-01, 1.1573e-03],
         [5.9149e+00, 9.0010e-02, 2.2100e-01, 9.3008e-04]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [1.1303e+00, 3.0908e+00, 4.1390e-01, 1.0847e-02],
         [3.2337e+00, 1.4184e+00, 4.7426e-01, 5.2687e-03],
         ...,
         [3.5525e+00, 2.6654e-01, 1.9232e-01, 1.9000e-03],
         [5.8183e+00, 2.3829e-01, 1.1745e-01, 1.8416e-03],
         [1.4650e+00, 3.8799e-


Train Diffusion: 100%|██████████| 21/21 [00:09<00:00,  2.29it/s]


In [11]:
train(devi, l_r, niter, piter, batch_size, ObsModel, csv_to_obs_df, 'AWB_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_C, SAWB_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB, get_CO2_AWB)


Train Diffusion:   0%|          | 0/21 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000005408.712109. Best norm loss value is: 54087.12109375.

x with CO2 means across time = tensor([[1.2560, 0.8153, 0.7611, 0.7870, 0.0044],
        [1.2215, 0.8094, 0.7810, 0.8063, 0.0045]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [1.0451e+00, 6.9476e-01, 6.2228e-01, 3.3647e-01, 3.1093e-03],
         [4.4421e+00, 6.8060e-01, 5.1105e-01, 4.2322e-01, 2.7536e-03],
         ...,
         [2.0989e+00, 9.6382e-01, 1.0069e+00, 6.3622e-01, 1.0143e-02],
         [1.2572e+00, 8.0679e-01, 6.0595e-01, 7.9381e-01, 6.4295e-03],
         [1.1817e+00, 5.5659e-01, 6.0622e-01, 4.8569e-01, 5.0505e-03]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [1.0451e+00, 4.0394e-01, 1.2394e+00, 1.0274e+00, 3.1561e-03],
         [5.9829e-01, 5.8341e-01, 7.3290e-01, 9.2884e-01, 3.2028e-03],
         ...,
         [5.4242e-01, 6.2279e-01, 8.0835e-01, 8.7780e-


Train Diffusion:  48%|████▊     | 10/21 [00:05<00:05,  1.87it/s]

Moving average norm loss at 10 iterations is: 49890.240234375. Best norm loss value is: 47914.375.

x with CO2 means across time = tensor([[6.2649e+00, 2.6968e-01, 2.6650e-01, 2.5763e-01, 8.7528e-04],
        [6.8594e+00, 2.8471e-01, 2.6516e-01, 2.5020e-01, 8.8102e-04]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [7.2108e-01, 9.9718e-01, 2.5273e-01, 3.2223e-01, 1.8666e-03],
         [4.5613e+00, 1.6978e+00, 2.9054e-01, 2.2222e-01, 2.8640e-03],
         ...,
         [2.0140e-01, 2.6326e-01, 2.5588e-01, 2.5909e-01, 1.3768e-03],
         [2.7784e+00, 2.7175e-01, 2.5221e-01, 2.5560e-01, 1.4271e-03],
         [8.0406e+00, 2.5663e-01, 3.0053e-01, 2.4372e-01, 1.6034e-03]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [1.2602e+00, 8.4927e+00, 2.6632e-01, 2.2015e-01, 3.8621e-03],
         [1.0497e+00, 6.6618e-01, 2.4745e-01, 2.3758e-01, 1.4946e-03],
         ...,
         [1.4


Train Diffusion:  95%|█████████▌| 20/21 [00:10<00:00,  1.92it/s]

Moving average ELBO loss at 20 iterations is: 361865.9109375. Best ELBO loss value is: 256057.765625.

x with CO2 means across time = tensor([[6.4640e+00, 3.5425e-01, 3.4912e-01, 3.4682e-01, 1.0818e-03],
        [6.9363e+00, 3.0394e-01, 3.1873e-01, 3.2076e-01, 9.8222e-04]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [1.6809e+00, 9.7793e+00, 9.1981e-02, 3.6455e-01, 1.3788e-03],
         [4.7182e+00, 1.5378e+00, 4.9012e-01, 1.5624e-01, 4.3188e-03],
         ...,
         [1.4203e+01, 9.7531e-02, 1.0203e-01, 6.4115e-01, 2.5757e-04],
         [8.1441e+00, 9.0842e-02, 1.7883e-01, 4.0323e-01, 4.0600e-04],
         [9.0880e-01, 1.2569e-01, 3.7613e-01, 1.0026e-01, 1.0191e-03]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [6.7466e-01, 1.2664e+00, 6.4590e-01, 9.5880e-02, 4.6802e-03],
         [1.4292e+00, 5.8674e-01, 1.1428e-01, 6.3325e-01, 6.8393e-04],
         ...,
         [


Train Diffusion: 100%|██████████| 21/21 [00:11<00:00,  1.86it/s]


In [12]:
train(devi, l_r, niter, piter, batch_size, ObsModel, csv_to_obs_df, 'AWB_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_SS, SAWB_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB, get_CO2_AWB)


Train Diffusion:   0%|          | 0/21 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000005403.253906. Best norm loss value is: 54032.5390625.

x with CO2 means across time = tensor([[1.0353, 0.6499, 0.7071, 0.7305, 0.0035],
        [1.0339, 0.6336, 0.6743, 0.6877, 0.0035]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [7.0236e-01, 6.6187e-01, 8.1107e-01, 6.9397e-01, 3.6174e-03],
         [7.3463e-01, 4.2432e-01, 4.6482e-01, 6.0069e-01, 1.8115e-03],
         ...,
         [1.0449e+00, 5.7373e-01, 5.9803e-01, 5.7713e-01, 4.9052e-03],
         [8.1919e-01, 5.4208e-01, 6.2937e-01, 8.4586e-01, 4.9855e-03],
         [8.6699e-01, 6.0229e-01, 5.6407e-01, 8.1816e-01, 5.0769e-03]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [8.1931e-01, 5.7317e-01, 5.5842e-01, 4.9875e-01, 2.5022e-03],
         [8.0978e-01, 4.3908e-01, 5.4561e-01, 8.4284e-01, 2.0943e-03],
         ...,
         [8.5266e-01, 6.3089e-01, 9.4436e-01, 1.2767e+0


Train Diffusion:  48%|████▊     | 10/21 [00:05<00:05,  1.88it/s]

Moving average norm loss at 10 iterations is: 50929.824609375. Best norm loss value is: 49003.734375.

x with CO2 means across time = tensor([[5.3592e+00, 2.6662e-01, 2.5397e-01, 2.4785e-01, 8.0053e-04],
        [5.5701e+00, 2.5852e-01, 2.5606e-01, 2.4696e-01, 8.0623e-04]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [5.7005e-01, 8.7194e+00, 2.5167e-01, 2.4331e-01, 3.6674e-03],
         [5.8734e-01, 5.4959e-01, 2.6776e-01, 2.1291e-01, 1.4049e-03],
         ...,
         [2.3086e+00, 2.2743e-01, 2.3523e-01, 2.2177e-01, 1.1356e-03],
         [8.7602e+00, 2.3925e-01, 2.5707e-01, 2.2882e-01, 1.3043e-03],
         [1.5834e+00, 2.5097e-01, 2.4775e-01, 3.2462e-01, 1.3431e-03]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [9.4791e-01, 3.7066e-01, 2.6349e-01, 2.5778e-01, 9.9591e-04],
         [2.0667e+00, 4.9159e+00, 2.4627e-01, 2.8361e-01, 3.4087e-03],
         ...,
         [


Train Diffusion:  95%|█████████▌| 20/21 [00:11<00:00,  1.84it/s]

Moving average ELBO loss at 20 iterations is: 44514151.0. Best ELBO loss value is: 24465070.0.

x with CO2 means across time = tensor([[6.9713e+00, 1.5647e-01, 1.8843e-01, 2.0438e-01, 4.0728e-04],
        [6.9467e+00, 1.7618e-01, 1.8951e-01, 2.1131e-01, 4.1105e-04]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [5.3093e-01, 2.5597e-01, 2.1572e-01, 2.4543e-01, 6.2523e-04],
         [4.8307e-01, 9.8662e-02, 2.2606e-01, 1.4875e-01, 2.9212e-04],
         ...,
         [1.1523e+01, 1.9186e-01, 1.5880e-01, 2.4410e-01, 7.0037e-04],
         [1.0182e+01, 1.6976e-01, 2.0034e-01, 3.0869e-01, 7.8768e-04],
         [8.1887e+00, 1.8183e-01, 1.3781e-01, 3.1549e-01, 6.1472e-04]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [9.0633e-01, 8.1454e+00, 1.3396e-01, 2.1607e-01, 1.9595e-03],
         [3.2090e+00, 2.0286e+00, 1.3756e-01, 2.2430e-01, 1.5292e-03],
         ...,
         [2.7458e


Train Diffusion: 100%|██████████| 21/21 [00:11<00:00,  1.77it/s]


In [13]:
train(devi, l_r, niter, piter, batch_size, ObsModel, csv_to_obs_df, 'AWB_ECA_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_ECA_C, SAWB_ECA_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB_ECA, get_CO2_AWB_ECA)


Train Diffusion:   0%|          | 0/21 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000005427.576563. Best norm loss value is: 54275.765625.

x with CO2 means across time = tensor([[0.8920, 0.7774, 0.7552, 0.7819, 0.0042],
        [0.9486, 0.7867, 0.7167, 0.7447, 0.0041]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [7.2549e-01, 3.2069e-01, 1.1981e+00, 1.0880e+00, 2.5419e-03],
         [1.5305e-01, 3.0032e-01, 8.0064e-01, 8.5252e-01, 1.9857e-03],
         ...,
         [7.6746e-01, 5.4399e-01, 6.0511e-01, 7.7564e-01, 4.7556e-03],
         [2.1066e-01, 4.9956e-01, 5.5649e-01, 8.1731e-01, 4.2904e-03],
         [4.6038e-01, 7.1920e-01, 1.2972e+00, 1.6051e+00, 1.0013e-02]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [7.2549e-01, 1.0396e+00, 7.9221e-01, 5.2866e-01, 4.8464e-03],
         [3.5317e-01, 1.4883e+00, 5.7240e-01, 5.3787e-01, 4.8293e-03],
         ...,
         [9.1298e-01, 6.0384e-01, 5.3678e-01, 5.1481e-01


Train Diffusion:  48%|████▊     | 10/21 [00:05<00:05,  1.98it/s]

Moving average norm loss at 10 iterations is: 51444.266015625. Best norm loss value is: 49462.609375.

x with CO2 means across time = tensor([[5.0088e+00, 3.3705e-01, 3.0428e-01, 2.5109e-01, 1.1792e-03],
        [4.9136e+00, 3.4291e-01, 3.0564e-01, 2.5222e-01, 1.1924e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [3.1828e-01, 1.2306e+00, 2.8469e-01, 2.7207e-01, 2.3210e-03],
         [6.4711e+00, 7.7233e-01, 3.9194e-01, 2.8593e-01, 2.4267e-03],
         ...,
         [1.7127e+00, 2.8591e-01, 3.2576e-01, 2.1613e-01, 1.7943e-03],
         [7.9914e+00, 3.1708e-01, 2.8095e-01, 2.6195e-01, 1.7689e-03],
         [1.5136e+00, 3.2554e-01, 2.8359e-01, 3.0132e-01, 1.8573e-03]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [1.3333e+00, 3.5046e+00, 3.6091e-01, 3.1399e-01, 4.3320e-03],
         [2.2572e-02, 1.1705e+00, 2.8866e-01, 2.4891e-01, 2.3839e-03],
         ...,
         [


Train Diffusion:  95%|█████████▌| 20/21 [00:10<00:00,  1.79it/s]

Moving average ELBO loss at 20 iterations is: 325215.63125. Best ELBO loss value is: 272315.4375.

x with CO2 means across time = tensor([[6.0124e+00, 2.6599e-01, 2.6930e-01, 2.9020e-01, 7.9936e-04],
        [5.9562e+00, 2.7115e-01, 2.7706e-01, 2.9163e-01, 8.1014e-04]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [1.7035e-01, 1.1752e+00, 4.7512e-01, 2.9466e-01, 3.5106e-03],
         [5.6522e+00, 4.7746e-01, 4.7171e-01, 2.2655e-01, 2.0048e-03],
         ...,
         [1.0885e+01, 3.8197e-01, 1.5206e-01, 3.5143e-01, 1.1756e-03],
         [8.0321e-01, 1.1148e-01, 3.2667e-01, 2.6987e-01, 8.0352e-04],
         [2.4365e+00, 1.0791e-01, 4.1409e-01, 2.4702e-01, 9.5041e-04]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [1.2270e+00, 3.7194e+00, 8.5562e-02, 2.1214e-01, 1.1037e-03],
         [2.8188e-02, 1.2788e+00, 1.5174e-01, 4.0458e-01, 1.3852e-03],
         ...,
         [2.52


Train Diffusion: 100%|██████████| 21/21 [00:11<00:00,  1.83it/s]


In [14]:
train(devi, l_r, niter, piter, batch_size, ObsModel, csv_to_obs_df, 'AWB_ECA_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_ECA_SS, SAWB_ECA_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB_ECA, get_CO2_AWB_ECA)


Train Diffusion:   0%|          | 0/21 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000005425.41836. Best norm loss value is: 54254.18359375.

x with CO2 means across time = tensor([[0.6578, 0.5881, 0.6606, 0.7005, 0.0032],
        [0.6638, 0.5831, 0.6693, 0.7079, 0.0032]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [4.8283e-01, 4.7027e-01, 3.6492e-01, 1.3216e+00, 1.5582e-03],
         [5.7209e-01, 2.0588e+00, 8.9802e-01, 9.8642e-01, 8.1071e-03],
         ...,
         [7.6418e-01, 3.2671e-01, 9.7955e-01, 5.4044e-01, 4.3084e-03],
         [6.5939e-01, 5.4677e-01, 6.9582e-01, 8.8959e-01, 5.3832e-03],
         [4.3020e-01, 6.1878e-01, 6.6024e-01, 1.2637e+00, 5.8034e-03]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [4.8283e-01, 4.7818e-01, 7.1722e-01, 5.2254e-01, 2.6032e-03],
         [5.0855e-01, 5.2132e-01, 5.0924e-01, 8.2572e-01, 2.2684e-03],
         ...,
         [6.5697e-01, 5.3610e-01, 8.3334e-01, 5.6054e-0


Train Diffusion:  48%|████▊     | 10/21 [00:05<00:06,  1.82it/s]

Moving average norm loss at 10 iterations is: 51034.75. Best norm loss value is: 48916.15234375.

x with CO2 means across time = tensor([[5.4663e+00, 2.1467e-01, 2.3311e-01, 2.3962e-01, 6.4981e-04],
        [5.4326e+00, 2.2381e-01, 2.3198e-01, 2.3273e-01, 6.5704e-04]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [4.9093e-01, 3.0656e-01, 2.0748e-01, 2.4425e-01, 7.0006e-04],
         [7.9355e-01, 6.1893e-01, 2.3291e-01, 2.4859e-01, 1.3506e-03],
         ...,
         [4.6169e-01, 2.2061e-01, 2.1094e-01, 2.5303e-01, 1.0093e-03],
         [1.3295e-02, 2.5364e-01, 2.3743e-01, 2.4144e-01, 1.2816e-03],
         [6.9354e+00, 3.0027e-01, 2.3528e-01, 1.9251e-01, 1.4895e-03]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [1.0441e+00, 2.2699e+00, 2.0365e-01, 2.5755e-01, 2.2176e-03],
         [3.4023e+00, 2.8316e+00, 3.0078e-01, 1.8857e-01, 3.5760e-03],
         ...,
         [1.056


Train Diffusion:  95%|█████████▌| 20/21 [00:11<00:00,  1.87it/s]

Moving average ELBO loss at 20 iterations is: 24206631.825. Best ELBO loss value is: 4091207.75.

x with CO2 means across time = tensor([[1.1329, 1.7979, 1.0189, 0.7594, 0.0070],
        [1.0180, 1.6806, 1.0421, 0.6684, 0.0069]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [5.9833e-01, 2.3029e-01, 2.1061e-01, 1.7076e-01, 5.6092e-04],
         [8.8877e-01, 3.3222e-01, 2.0933e-01, 1.2794e+00, 7.8272e-04],
         ...,
         [1.3318e-01, 3.1752e+00, 1.0514e+00, 1.1115e-01, 1.9831e-02],
         [1.0038e+00, 3.1910e-01, 1.5014e-01, 1.4607e-01, 1.0347e-03],
         [1.0770e+00, 8.0489e-02, 2.0421e-01, 9.5852e-01, 4.1418e-04]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [1.2822e+00, 1.6852e+00, 3.5692e+00, 3.2954e+00, 1.6026e-02],
         [2.9153e+00, 3.3389e+00, 3.0868e+00, 1.0253e-01, 2.4081e-02],
         ...,
         [1.1824e+00, 3.1960e-01, 2.6334e-01, 8.6326e-01, 1.65


Train Diffusion: 100%|██████████| 21/21 [00:11<00:00,  1.76it/s]
